In [4]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *

appName = "Final Project - Task 2"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than rely on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
#sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 14:44:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/24 14:44:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
def read_db(db_name):
    df = spark.read.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/postgres")\
    .option("dbtable", db_name)\
    .option("user", "jc")\
    .option("password", "")\
    .option("Driver", "org.postgresql.Driver")\
    .load()
    return df

In [7]:
def read_db(db_name):
    df = spark.read.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/postgres")\
    .option("dbtable", db_name)\
    .option("user", "xintongwu")\
    .option("password", "")\
    .option("Driver", "org.postgresql.Driver")\
    .load()
    return df

In [8]:
player_dfs = {}
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

player_df = read_db("fifa.player_all")
for year in years:
    player_dfs[year] = player_df[player_df['year'] == year]

## 1. List the x players who achieved average highest improvement across all skillsets.

In [9]:
skillsets = { "pace", "shooting", "passing", "dribbling", "defending", "physic", "attacking_crossing", 
             "attacking_finishing", "attacking_heading_accuracy", "attacking_short_passing", "attacking_volleys", 
             "skill_dribbling", "skill_curve", "skill_fk_accuracy", "skill_long_passing", "skill_ball_control", 
             "movement_acceleration", "movement_sprint_speed", "movement_agility", "movement_reactions", 
             "movement_balance", "power_shot_power", "power_jumping", "power_stamina", "power_strength", 
             "power_long_shots", "mentality_aggression", "mentality_interceptions", "mentality_positioning", 
             "mentality_vision", "mentality_penalties", "mentality_composure", "defending_marking_awareness", 
             "defending_standing_tackle", "defending_sliding_tackle", "goalkeeping_diving", "goalkeeping_handling", 
             "goalkeeping_kicking", "goalkeeping_positioning", "goalkeeping_reflexes", "goalkeeping_speed" 
            }

def task_II_1(x, from_year='2015', to_year='2022'):
    def map_improvement(row):
        imp = 0
        for col in skillsets:
            target_col = 'target_' + col
            if col in row and row[col] and target_col in row and row[target_col]:
                imp += row[target_col] - row[col]
        return (imp, (row['sofifa_id'], row['short_name']))
    from_df = player_dfs[from_year]
    to_df = player_dfs[to_year]
    for col in skillsets:
        to_df = to_df.withColumnRenamed(col, "target_" + col)
    compare_df = from_df.join(to_df, on='sofifa_id', how='inner')
    results = compare_df.rdd.map(map_improvement).sortByKey(ascending=False).take(x)
    improvement, players = zip(*results)
    return players, improvement

In [10]:
# Example
x = 10
from_year, to_year = '2015', '2022'
top_imp_players, _ = task_II_1(x, from_year, to_year)

print(f"Top {x} players achieved highest improvement from {from_year} to {to_year} are:")
for player in top_imp_players:
    print(player[1], f"(id:{player[0]})")

/Users/xintongwu/miniforge3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


Top 10 players achieved highest improvement from 2015 to 2022 are:
K. Laimer (id:225375)
S. Milinković-Savić (id:223848)
M. Al Buraik (id:221370)
D. Calvert-Lewin (id:221479)
R. Andrich (id:212242)
O. Watkins (id:221697)
R. Steffen (id:210537)
P. Ng (id:223905)
R. Skov (id:215449)
Alex Berenguer (id:225201)


## 2. What are the y clubs that have largest number of players with contracts ending in 2021?

In [8]:
def task_II_2(y, valid_until="2021", at_year="2021"):
    df = player_dfs[at_year]
    counts = df[df['club_contract_valid_until'] == valid_until].groupBy('club_team_id', 'club_name').count()
    top_y = counts.sort('count', ascending=False).take(y)
    return [club['club_name'] for club in top_y]

In [9]:
print("In year 2018, the 5 clubs with largest number of players with contracts ending in 2021 are", 
      ", ".join(task_II_2(5, at_year='2018')))

In year 2018, the 5 clubs with largest number of players with contracts ending in 2021 are Spartak Moskva, VfL Wolfsburg, Sevilla FC, Newcastle United, Borussia Mönchengladbach


## 3. List the z clubs with largest number of players in the dataset where z >= 5

In [10]:
def task_II_3(z, year='2022'):
    assert z >= 5, "z must be at least 5"
    df = player_df if year is None else player_dfs[year]
    sorted_counts = df.dropna(subset=['club_team_id']).groupBy('club_team_id', 'club_name').count().sort('count', ascending=False).collect()
    if sorted_counts[-1]['count'] == sorted_counts[0]['count']:
        print('All teams have the same number of players:', sorted_counts[-1]['count'])
    teams = [team['club_name'] for team in sorted_counts[:min((z, len(sorted_counts)))]]
    return teams

def task_II_3_inclusive(z, year='2022'):
    assert z >= 5, "z must be at least 5"
    df = player_df if year is None else player_dfs[year]
    sorted_counts = df.dropna(subset=['club_team_id']).groupBy('club_team_id', 'club_name').count().sort('count', ascending=False).collect()
    if sorted_counts[-1]['count'] == sorted_counts[0]['count']:
        print('All teams have the same number of players:', sorted_counts[-1]['count'])
    teams, n = [], 0
    while z - n > 0:
        count = sorted_counts[n]['count']
        while n < len(sorted_counts) and sorted_counts[n]['count'] >= count:
            teams.append(sorted_counts[n]['club_name'])
            n += 1
    return teams

In [11]:
print("In year 2022,\n")
print("Top 7 clubs with largest number of players are")
print(", ".join(task_II_3(7)))
print()
print("The clubs that have top 7 largest number of players include, including ties:")
print(", ".join(task_II_3_inclusive(7)))

In year 2022,

Top 7 clubs with largest number of players are
Levante Unión Deportiva, Arsenal, Wolverhampton Wanderers, Granada CF, Newcastle United, RCD Espanyol de Barcelona, Manchester United

The clubs that have top 7 largest number of players include, including ties:
Levante Unión Deportiva, Arsenal, Wolverhampton Wanderers, Granada CF, Newcastle United, RCD Espanyol de Barcelona, Manchester United, Villarreal CF, Chelsea, ESTAC Troyes, Burnley, Real Madrid CF, Borussia Mönchengladbach, RCD Mallorca, Brentford, Paris Saint-Germain, Southampton, Olympique de Marseille, FC Barcelona, CA Osasuna, Leicester City, Sevilla FC, Crystal Palace, Everton, Norwich City, Valencia CF, TSG Hoffenheim, Venezia FC, RC Celta de Vigo, West Ham United, Real Betis Balompié, Brighton & Hove Albion, Liverpool, Genoa, Tottenham Hotspur, VfB Stuttgart


## 4. What is the most frequent nation_position and team_position in the dataset? (list the most popular for each)

In [12]:
def task_II_4(year=None, columns=['nation_position', 'club_position']):
    def find_most_frequent(df, column):
        counts = df.dropna(subset=[column]).groupBy(column).count()
        top_count = counts.sort('count', ascending=False).take(1)[0]['count']
        rows = counts[counts['count'] == top_count].collect()
        top_rows = [row[column] for row in rows]
        return top_rows
    df = player_df if year is None else player_dfs[year]
    top_results = {}
    for col in columns:
        top_results[col] = find_most_frequent(df, col)
    return top_results

In [13]:
print("Most frequent positions in the entire dataset:")
print(task_II_4())
print("Most frequent positions in year 2022:")
print(task_II_4("2022"))

Most frequent positions in the entire dataset:
{'nation_position': ['SUB'], 'club_position': ['SUB']}
Most frequent positions in year 2022:
{'nation_position': ['SUB'], 'club_position': ['SUB']}


## 5. What is the most popular nationality for the players in the dataset?

In [21]:
def task_II_5(year=None):
    df = player_df if year is None else player_dfs[year]
    counts = df.dropna(subset=['nationality_id']).groupBy('nationality_id', 'nationality_name').count()
    top_count = counts.sort('count', ascending=False).take(1)[0]['count']
    top_rows = counts[counts['count'] == top_count].collect()
    top_nations = [row['nationality_name'] for row in top_rows]
    return top_nations if len(top_nations) > 1 else top_nations[0]

In [22]:
print("The most popular player nationality in the dataset is", task_II_5())

for year in years:
    print("The most popular player nationality in year", year, "is", task_II_5(year))

The most popular player nationality in the dataset is England
The most popular player nationality in year 2015 is England
The most popular player nationality in year 2016 is England
The most popular player nationality in year 2017 is England
The most popular player nationality in year 2018 is England
The most popular player nationality in year 2019 is England
The most popular player nationality in year 2020 is England
The most popular player nationality in year 2021 is England
The most popular player nationality in year 2022 is England
